In [1]:
import os
import pickle
import pandas as pd
import mygene

clusters=[]
files = os.listdir('/home/jain/clusters/0911_0.85')
print(files)
os.chdir('/home/jain/clusters/0911_0.85')
log_all = pd.read_csv('https://ftp.mcs.anl.gov/pub/candle/public/benchmarks/Pilot1/normal-tumor/matched_normal_samples.FPKM-UQ.log-transformed.csv', sep=',')
import mygene
mg = mygene.MyGeneInfo()

['cf_class_1_cluster0.pkl', 'cf_class_0_cluster0.pkl', 'cf_class_0_cluster1.pkl', 'cf_class_1_cluster1.pkl']


- Perturbation vectors were obtained by running Counterfactual analysis on NT3 dataset
- These are the files (pkl above). Each file represents 1 cluster of perturbation vectors. Using KMeans with 2D PCA.
- Inside each file is the centroid, positive indices (indices whos threshold is > 0.9max), negative indices (indices whos threshold is < -0.9max), all the sample indices (from the 1100 dataset) that belong to that particular cluster.
- The file names correspond to the cf class (0 or 1) and the cluster index.
- Here we try to do a literature review of the threshold genes for cancer
- The various types of cancer tumors used in our dataset are here: https://ftp.mcs.anl.gov/pub/candle/public/benchmarks/Pilot1/normal-tumor/matched_normal_samples.metadata.tsv

In [2]:
gene_all = list(log_all.columns.values)


In [3]:
print(len(gene_all))

60484


In [4]:
gene_all=gene_all[1:]
gene_all[0]

'ENSG00000000003.13'

In [5]:
gene_all[1]

'ENSG00000000005.5'

In [6]:
gene_all[2]

'ENSG00000000419.11'

In [7]:
os.chdir('/home/jain/clusters/0911_0.85')
cluster_genes=[]
total_positive=0
total_negative=0
for f in files:
    clusters = pickle.load(open(f, "rb"))
    print("Cluster: ", f)
    print("sample ids:", clusters['sample indices in this cluster'])
    print("+ve",clusters['positive threshold indices'])
    print("-ve",clusters['negative threshold indices'])
    a=clusters['positive threshold indices']
    b=clusters['negative threshold indices']
    print("\n\n")

    for bb in b:
        for i in bb:
            total_negative+=1
            print("print", i, type(i))
            cluster_genes.append(i)
    for aa in a:
        for j in aa:
            total_positive+=1
            print("print", j, type(j))
            cluster_genes.append(j)
cluster_genes

Cluster:  cf_class_1_cluster0.pkl
sample ids: [   8   13   22   23   24   29   32   43   46   47   52   59   60   66
   71   78   82   84   87   88   92   96   97  100  106  108  109  110
  111  119  122  125  129  131  139  151  163  166  167  171  177  185
  196  204  208  210  212  216  227  232  234  242  246  247  248  252
  256  258  275  278  285  302  306  307  313  315  318  320  325  329
  338  343  346  349  355  366  367  369  376  389  392  396  397  400
  402  404  406  411  412  416  417  423  426  427  431  442  445  447
  449  452  475  492  494  498  500  504  507  508  509  520  523  527
  535  538  541  542  552  554  555  558  561  562  566  567  568  569
  578  583  584  586  587  592  596  599  617  621  624  625  627  648
  650  652  654  655  659  661  664  669  670  672  673  675  682  684
  686  688  691  699  701  704  708  718  723  726  732  738  742  746
  750  752  753  763  767  773  778  783  784  788  789  795  798  801
  804  805  808  816  829  834 

[9603, 17114, 32077, 12660, 58261, 12122, 8618, 9603, 14270, 32077, 44932]

In [8]:
total_positive

0

In [9]:
total_negative

11

In [10]:
gene_names = []
# 9603 32077 appears twice? 
cluster_genes=list(set(cluster_genes))
for g in cluster_genes:
    split=gene_all[g].split(".")
    gene_names.append(split[0])
    
    
cluster_genes

[9603, 44932, 8618, 32077, 12660, 58261, 17114, 12122, 14270]

In [11]:
geneSyms = mg.querymany(gene_names , scopes='ensembl.gene', fields='symbol', species='human')
gene_names

querying 1-9...done.
Finished.


['ENSG00000152952',
 'ENSG00000257113',
 'ENSG00000144771',
 'ENSG00000232995',
 'ENSG00000169704',
 'ENSG00000278967',
 'ENSG00000196367',
 'ENSG00000167543',
 'ENSG00000177239']

In [12]:
geneSyms

[{'query': 'ENSG00000152952',
  '_id': '5352',
  '_score': 21.653376,
  'symbol': 'PLOD2'},
 {'query': 'ENSG00000257113', '_id': 'ENSG00000257113', '_score': 22.567783},
 {'query': 'ENSG00000144771',
  '_id': '57408',
  '_score': 22.50901,
  'symbol': 'LRTM1'},
 {'query': 'ENSG00000232995',
  '_id': '8490',
  '_score': 21.656713,
  'symbol': 'RGS5'},
 {'query': 'ENSG00000169704',
  '_id': '2815',
  '_score': 21.653376,
  'symbol': 'GP9'},
 {'query': 'ENSG00000278967', '_id': 'ENSG00000278967', '_score': 22.567783},
 {'query': 'ENSG00000196367',
  '_id': '8295',
  '_score': 22.042913,
  'symbol': 'TRRAP'},
 {'query': 'ENSG00000167543',
  '_id': '90313',
  '_score': 22.04625,
  'symbol': 'TP53I13'},
 {'query': 'ENSG00000177239',
  '_id': '11253',
  '_score': 21.987478,
  'symbol': 'MAN1B1'}]

In [13]:
symbols = [i['symbol'] for i in geneSyms if 'symbol' in i]
symbols

['PLOD2', 'LRTM1', 'RGS5', 'GP9', 'TRRAP', 'TP53I13', 'MAN1B1']

In [14]:
# There are only 6 genes and all on the negative threshold side
# This list is probably just saying that most of the time these genes are very different (high or low) compared to other 60484
geneSyms[:][0] 
# appeared twice in our list

{'query': 'ENSG00000152952',
 '_id': '5352',
 '_score': 21.653376,
 'symbol': 'PLOD2'}

https://www.proteinatlas.org/ENSG00000152952-PLOD2  One major killer gene: Prognostic marker in renal cancer (unfavorable), liver cancer (unfavorable), lung cancer (unfavorable), cervical cancer (unfavorable) and stomach cancer (unfavorable)

In [15]:
geneSyms[:][1]

{'query': 'ENSG00000257113', '_id': 'ENSG00000257113', '_score': 22.567783}

https://www.proteinatlas.org/ENSG00000144771-LRTM1/pathology urothelial cancer

In [16]:
geneSyms[:][2]
# appeared twice in our list

{'query': 'ENSG00000144771',
 '_id': '57408',
 '_score': 22.50901,
 'symbol': 'LRTM1'}

In conclusion, the present study showed that RGS5 played an inhibitory role in human lung cancer cells through induction of apoptosis. Furthermore, RGS5 enhanced the cytotoxic effect of radiation in the human lung cancer cells. Our results indicated that RGS5 may be a potential target for cancer therapy.

In [17]:
geneSyms[:][3]

{'query': 'ENSG00000232995',
 '_id': '8490',
 '_score': 21.656713,
 'symbol': 'RGS5'}

https://www.proteinatlas.org/ENSG00000143344-RGL1/pathology no specific type of cancer but graph shows can be present at elevated levels for several types of cancer - part of cluster 0

In [18]:
geneSyms[:][4]

{'query': 'ENSG00000169704',
 '_id': '2815',
 '_score': 21.653376,
 'symbol': 'GP9'}

https://www.proteinatlas.org/ENSG00000167543-TP53I13/pathology
Renal cancer

In [19]:
geneSyms[:][5]

{'query': 'ENSG00000278967', '_id': 'ENSG00000278967', '_score': 22.567783}

Novel MAN1B1 variants were identified in patients with facial dysmorphism, hypotonia, truncal obesity and in some, behavioural problems. Results show that MAN1B1 expression was higher in bladder cancer (BC) tissues than those in normal tissues. Its overexpression was associated with poor prognosis.

In [20]:
meta = pd.read_csv('https://ftp.mcs.anl.gov/pub/candle/public/benchmarks/Pilot1/normal-tumor/matched_normal_samples.metadata.tsv', sep=',')

In [109]:
meta

,aeec3005-25d7-45fc-a9ff-014d57960216\tProstate\tProstate Adenocarcinoma\tSolid Tissue Normal\t76a194e5-ba36-4e81-b0b4-6c22a69fe9b5.FPKM-UQ.txt\tPrimary Tumor\t26b9d14a-da14-4062-8494-fb87b04e9d7b.FPKM-UQ.txt
0,c8fb94dd-c535-48f4-9419-aac2bf47bbb4\tUterus\t...
1,de206681-3e27-4d4f-8669-aefb12b5b1e3\tHead and...
2,c32e04c5-2760-4a11-a7a9-ded1f4eb3a8c\tLung\tLu...
3,98dea82b-46f9-4b77-be8a-06669bcf731b\tKidney\t...
4,31da07bb-6752-49a3-a7ea-5427c04a9458\tBladder\...
...,...
720,a935daeb-1efd-4113-a614-a5e4d446c72b\tEsophagu...
721,dd3bfb26-b534-4917-9c4d-9fe7b6477762\tBreast\t...
722,424de829-9efd-4689-ac5d-24485a5d5ecf\tThyroid\...
723,8ad78c4f-c84c-4f9e-88c9-42a05eb65498\tProstate...
